# Adding Coordinates to Neighborhood DataFrame

In [4]:
# adding Neighborhood DataFrame
import requests
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import pandas as pd
import numpy as np
import ssl 

# Disable SSL certificate verification
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url)
html_b = BeautifulSoup(html.text, 'html.parser')
value =[]
df = pd.DataFrame(columns = ['Postcode', 'Borough', 'Neighbourhood'])
for i in html_b.find(class_ = "wikitable sortable" ).select('tr'):
    value.append(i.get_text())

for b in value:
    x = b.splitlines()
    if x[2] != 'Not assigned':
        if x[3] == 'Not assigned':
            x[3] = x[2]  
        if df.loc[df['Postcode'] == x[1]].empty is False:
            indx = df.index[df['Postcode'] == x[1]].tolist()
            name = (df.iloc[indx]['Neighbourhood'].tolist())
            name = name[0]+', ' + x[3]
            df['Neighbourhood'].iloc[indx] = name
        else: 
            df = df.append({'Postcode' : x[1], 'Borough' : x[2],  'Neighbourhood': x[3]},  ignore_index=True)      
    
df = df.drop([0])   

In [5]:
#adding extra Columns into Dataframe for coordinates
df['Latitude'] = ''
df['Longitude'] = ''

In [6]:
#importing Coordinate CSV file
url_CSV = 'https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
cord = pd.read_csv(url_CSV)

In [7]:
#adding coordinate values to Neighborhood DataFrame
p=0
while p < len(cord.index.values):                                          #sets loop for number of rows
    post_code = cord.loc[p]['Postal Code']                              #get post code
    indx = df.index[df['Postcode'] == post_code].tolist()       #finds index values of the post code in Neighborhood DataFrame
    df.loc[indx[0]]['Latitude'] = cord.loc[p]['Latitude']          #add Latidute to the df
    df.loc[indx[0]]['Longitude'] = cord.loc[p]['Longitude']     #add Longitude to the df
    p += 1

In [8]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
1,M3A,North York,Parkwoods,43.7533,-79.3297
2,M4A,North York,Victoria Village,43.7259,-79.3156
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6543,-79.3606
4,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7185,-79.4648
5,M7A,Queen's Park,Queen's Park,43.6623,-79.3895
